In [1]:
### Dependencies

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
import re

### This notebook is for extracting the data from the original kml file and outputing a JSON that will be easy to use with Google Maps API



In [2]:
## LOAD BLOCK###
#### Load data from kml file exported from Google Earth

file_path = ('data/kml/ballparks.kml') # file path to kml file

with open(file_path) as file:

    xml_data = file.read()



# Initialize soup variables for parsing file
soup = BeautifulSoup(xml_data, 'xml')
folders = soup.Document.Folder
list = soup.Document.Folder.find_all('Folder')

## Create a dataframe to hold the data parsed from xml
df = pd.DataFrame(columns=['field', 'foul', 'fop'])

failed = []


In [3]:
#### EXTRACTION BLOCK ####
#### Extract data from kml file

# Create an empty list to store the rows to append to the DataFrame
rows = []

# Loop through the folders and extract the data
for folder in list:
    try:
        field_name = folder.find('name').text
        foul = folder.find_all('coordinates')[0].text
        fop = folder.find_all('coordinates')[1].text

        row = {
            'field': field_name,
            'foul': foul,
            'fop': fop
        }

        rows.append(row)

    except Exception as e:
        # Add name of folder to a list of failed folders
        failed.append(folder.find('name').text)
        print(f"Error processing folder: {folder.find('name').text}. Error message: {str(e)}")

# Convert the list of rows to a DataFrame
df = pd.DataFrame(rows)

# Print a list of failed folders
print(f"Failed to process {len(failed)} folders: {', '.join(failed)}")


Error processing folder: Progressive Field - Cleveland - MLB. Error message: list index out of range
Failed to process 1 folders: Progressive Field - Cleveland - MLB


In [4]:
# Define a dictionary that maps level indicators to levels and size factors
level_dict = {
    'International': 'international',
    'Major Leagues': 'mlb', 
    'Professional': 'pro', 
    'College': 'college', 
    'High School': 'high_school',
    'Youth': 'youth',
    
}

In [5]:
# Create a copy of the original DataFrame
df_cleaned = df.copy()

# Remove new line and space characters from coordinates
df_cleaned = df_cleaned.replace(r'\n','', regex=True) 
df_cleaned = df_cleaned.replace(r'\t','', regex=True) 

# Drop any duplicate rows
df_cleaned = df_cleaned.drop_duplicates(subset=['field'], keep='first')

# Drop any rows with empty fields
df_cleaned = df_cleaned[(df_cleaned != 0).all(1)]

# Define the regex patterns for each level
re_mlb = re.compile(r'mlb', re.IGNORECASE)
re_pro = re.compile(r'pro|semi[-\s]*pro', re.IGNORECASE)
re_college = re.compile(r'college', re.IGNORECASE)
re_high_school = re.compile(r'high school|hs', re.IGNORECASE)  # Include the abbreviation 'hs'
re_youth = re.compile(r'youth', re.IGNORECASE)
re_muni = re.compile(r'muni', re.IGNORECASE)
re_international = re.compile(r'international', re.IGNORECASE)

# Define a function to classify the fields based on the regex patterns
def classify_field(field_name):
    if re_mlb.search(field_name):
        return 'Major League'
    elif re_pro.search(field_name):
        return 'Professional'
    elif re_college.search(field_name):
        return 'College'
    elif re_high_school.search(field_name):
        return 'High School'
    elif re_youth.search(field_name):
        return 'Youth'
    elif re_muni.search(field_name):
        return 'State / County / Municipal'
    elif re_international.search(field_name):
        return 'International'
    else:
        return 'Unknown'

# Apply the classify_field function to the 'field' column
df_cleaned['level'] = df_cleaned['field'].apply(classify_field)

# Clean up the 'field' column by removing the level indicator and any trailing '-' characters
level_regex = r'\s*(%s)\s*' % '|'.join(re.escape(level) for level in level_dict.values())
df_cleaned['field'] = df_cleaned['field'].str.replace(level_regex, '', regex=True, flags=re.IGNORECASE)
df_cleaned['field'] = df_cleaned['field'].str.replace(r'-\s*$', '', regex=True)

# Rename field column to park_name to avoid confusion down the line
df_cleaned = df_cleaned.rename(columns={'field': 'park_name'})



In [6]:
df_cleaned.head()

## print a list of all the values in the level column
print(df_cleaned['level'].unique())

## Print the two the headers and two rows of data to a txt file for reference


# with open('data/rows.txt', 'w') as f:
#     f.write(df_cleaned.iloc[0].to_string())
#     f.write(df_cleaned.iloc[1].to_string())
#     f.write(df_cleaned.iloc[14].to_string())
    

# print(df_cleaned.iloc[15].to_string())


['Professional' 'College' 'International' 'Major League' 'Unknown'
 'High School' 'Youth' 'State / County / Municipal']


In [7]:
##### Clean up polygon data and create a new home_plate column

def parse_coordinates(coord_string):
    coords = coord_string.split()
    parsed_coords = [tuple(map(float, coord.split(',')[:2])) for coord in coords]
    return parsed_coords

# Create a new column for the home_plate location using the first set of coordinates in the 'fop' column
df_cleaned['home_plate'] = df_cleaned['fop'].apply(lambda x: parse_coordinates(x)[0])

# Apply the parse_coordinates function to the 'foul' and 'fop' columns
df_cleaned['foul'] = df_cleaned['foul'].apply(parse_coordinates)
df_cleaned['fop'] = df_cleaned['fop'].apply(parse_coordinates)


In [8]:
## Doesn't seem to be returning useful data - will need to revisit

# def determine_direction(coordinates):
#     num_points = len(coordinates)
#     if num_points < 2:
#         return "Not enough points"

#     # Get the latitude (y-coordinate) values
#     latitudes = [point[1] for point in coordinates]

#     # Check the change in latitude values
#     increasing_latitudes = all(latitudes[i] <= latitudes[i+1] for i in range(num_points-1))
#     decreasing_latitudes = all(latitudes[i] >= latitudes[i+1] for i in range(num_points-1))

#     if increasing_latitudes:
#         return "Left Field"
#     elif decreasing_latitudes:
#         return "Right Field"
#     else:
#         return "Collinear"

# # Apply the determine_direction function to the 'fop' and 'foul' columns
# df_cleaned['fop_direction'] = df_cleaned['fop'].apply(determine_direction)
# df_cleaned['foul_direction'] = df_cleaned['foul'].apply(determine_direction)

# # Print the updated DataFrame
# print(df_cleaned)



In [9]:
df_cleaned.head()

## Value counts for the fop_direction column
df_cleaned['level'].value_counts()

## Value counts for the foul_direction column
# df_cleaned['foul_direction'].value_counts()

df_cleaned.head()




,park_name,foul,fop,level,home_plate
0,Day Air Ballpark - Dayton Dragons,"[(-84.18557, 39.7642091), (-84.1849444, 39.764...","[(-84.18557, 39.7642091), (-84.1846064, 39.763...",Professional,"(-84.18557, 39.7642091)"
1,Parkview Field - Fort Wayne TinCaps,"[(-85.1429982, 41.0741133), (-85.1432208, 41.0...","[(-85.1429982, 41.0741133), (-85.1418777, 41.0...",Professional,"(-85.1429982, 41.0741133)"
2,Classic Park- Lake County Captains,"[(-81.4362001, 41.6407581), (-81.4362008, 41.6...","[(-81.4362001, 41.6407581), (-81.4350347, 41.6...",Professional,"(-81.4362001, 41.6407581)"
3,ABC Supply Stadium - Beloit Sky Carp,"[(-89.0406651, 42.4971349), (-89.0408006, 42.4...","[(-89.0406651, 42.4971349), (-89.0394729, 42.4...",Professional,"(-89.0406651, 42.4971349)"
4,Veterans Memorial Stadium - Cedar Rapids Kernels,"[(-91.6867962, 41.9677456), (-91.6867801, 41.9...","[(-91.6867962, 41.9677456), (-91.6856006, 41.9...",Professional,"(-91.6867962, 41.9677456)"


In [10]:
### Calculate Areas of Foul and FOP

from shapely.geometry import Polygon
import pyproj


#### Functions to calculate the area of a polygon
def utm_zone_number(lat, lon):
    return int((lon + 180) / 6) + 1

def utm_epsg_code(lat, lon):
    zone_number = utm_zone_number(lat, lon)
    if lat >= 0:
        return 32600 + zone_number
    else:
        return 32700 + zone_number

def calculate_area(coords):
    # Create a Polygon object from the coordinates
    polygon = Polygon(coords)

    # Get the appropriate UTM EPSG code
    epsg_code = utm_epsg_code(*coords[0])

    # Create a transformer for converting coordinates to UTM
    transformer = pyproj.Transformer.from_crs(
        pyproj.CRS("EPSG:4326"),  # WGS 84 (latitude and longitude)
        pyproj.CRS(f"EPSG:{epsg_code}"),  # UTM zone
        always_xy=True
    )

    # Convert the coordinates to UTM
    coords_utm = [transformer.transform(*coord) for coord in coords]

    # Create a Polygon object from the UTM coordinates
    polygon_utm = Polygon(coords_utm)

    # Calculate the area in square meters
    area_sqm = polygon_utm.area

    # Convert the area to square feet (1 square meter = 10.764 square feet)
    area_sqft = area_sqm * 10.764

    return area_sqft


### Call Function and add to dataframe
df_cleaned['foul_area_sqft'] = df_cleaned['foul'].apply(calculate_area)
df_cleaned['fop_area_sqft'] = df_cleaned['fop'].apply(calculate_area)

## Calculate the total area of the field and the ratio of foul area to field area
df_cleaned['field_area_sqft'] = df_cleaned['foul_area_sqft'] + df_cleaned['fop_area_sqft']
## Percentage foul area
df_cleaned['foul_area_per'] = df_cleaned['foul_area_sqft'] / df_cleaned['field_area_sqft']
## Fair to Foul Ratio
df_cleaned['fair_to_foul'] = df_cleaned['fop_area_sqft'] / df_cleaned['foul_area_sqft']


In [11]:
#### SIMPLE DISTANCE CALCULATION USING GEOPY ####

## Returns a list of distances from home plate to each outfield coordinate
from geopy.distance import great_circle

def calculate_distances(home_plate, outfield_coords):
    def is_same_point(point1, point2, tolerance=1e-6):
        return abs(point1[0] - point2[0]) < tolerance and abs(point1[1] - point2[1]) < tolerance

    home_plate_lat_lon = (home_plate[1], home_plate[0])
    distances = [
        round(great_circle(home_plate_lat_lon, (coord[1], coord[0])).feet)
        for coord in outfield_coords
        if not is_same_point(home_plate, coord)
    ]
    return distances


In [12]:
### Run Distance Functions and add to dataframe

# Calculate distances for each row
df_cleaned['distances'] = df_cleaned.apply(lambda row: calculate_distances(row['home_plate'], row['fop']), axis=1)

# Calculate max, min, and average distances for each row
df_cleaned['max_distance'] = df_cleaned['distances'].apply(max)
df_cleaned['min_distance'] = df_cleaned['distances'].apply(min)
df_cleaned['avg_distance'] = df_cleaned['distances'].apply(lambda distances: sum(distances) / len(distances))


In [13]:
#### Calculate Ranks for each field
### Grouped by level

def rank_fields(df):
    # Calculate the rank for each category
    df['max_distance_rank'] = df['max_distance'].rank(ascending=False, method='min')
    df['min_distance_rank'] = df['min_distance'].rank(ascending=False, method='min')
    df['avg_distance_rank'] = df['avg_distance'].rank(ascending=False, method='min')
    df['field_area_rank'] = df['field_area_sqft'].rank(ascending=False, method='min')
    df['ratio_rank'] = df['fair_to_foul'].rank(ascending=False, method='min')

    return df

# Group the DataFrame by level and apply the rank_fields function to each group
df_ranked = df_cleaned.groupby('level').apply(rank_fields)

# Reset the index to get the original DataFrame structure
df_ranked.reset_index(drop=True, inplace=True)


In [14]:
## Show samples of the data from each level

df_ranked[df_ranked['level'] == 'high_school'].head(10)

# df_ranked.head()

,park_name,foul,fop,level,home_plate,foul_area_sqft,fop_area_sqft,field_area_sqft,foul_area_per,fair_to_foul,distances,max_distance,min_distance,avg_distance,max_distance_rank,min_distance_rank,avg_distance_rank,field_area_rank,ratio_rank


In [15]:
## Not sure if this is useful - will need to revisit

# import numpy as np

# def calculate_orientation(coordinates):
#     num_points = len(coordinates)
#     if num_points < 3:
#         return "Not enough points"

#     signed_area = 0

#     for i in range(num_points):
#         x1, y1 = coordinates[i]
#         x2, y2 = coordinates[(i + 1) % num_points]
#         signed_area += (x2 - x1) * (y2 + y1)

#     if signed_area > 0:
#         return "Counterclockwise"
#     elif signed_area < 0:
#         return "Clockwise"
#     else:
#         return "Collinear"

# # Apply the calculate_orientation function to the 'fop' and 'foul' columns
# df_cleaned['fop_orientation'] = df_cleaned['fop'].apply(calculate_orientation)
# df_cleaned['foul_orientation'] = df_cleaned['foul'].apply(calculate_orientation)

# # Print the updated DataFrame
# print(df_cleaned)



In [16]:
# #### Show value counts od orientation columns
# print(df_cleaned['fop_orientation'].value_counts())
# print(df_cleaned['foul_orientation'].value_counts())

In [17]:
# ### Print the DataFrame to a string for reference

# df_cleaned_str = df_cleaned.head().to_string()
# print(df_cleaned_str)

# # Output the string to a txt file for reference
# with open('data/df_cleaned_str.txt', 'w') as f:
#     f.write(df_cleaned_str)

    




In [18]:
#### Orienting the map to the home plate location ####

### Find the center of the field
def calculate_centroid(coords):
    x_coords = [coord[0] for coord in coords]
    y_coords = [coord[1] for coord in coords]
    centroid_x = sum(x_coords) / len(coords)
    centroid_y = sum(y_coords) / len(coords)
    return (centroid_x, centroid_y)


## Find the bearing between the home plate and the center of the field
import math

def calculate_bearing(point1, point2):
    lat1, lon1 = math.radians(point1[1]), math.radians(point1[0])
    lat2, lon2 = math.radians(point2[1]), math.radians(point2[0])

    d_lon = lon2 - lon1

    x = math.cos(lat2) * math.sin(d_lon)
    y = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(d_lon)

    bearing = math.degrees(math.atan2(x, y))
    bearing = (bearing + 360) % 360  # Normalize the bearing to the range [0, 360)

    return bearing

### Function to classify direction in laymans terms North, South, East, West, ect
def degrees_to_cardinal_direction(degrees):
    directions = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW', 'N']
    index = round(degrees / 45)
    return directions[index]



In [19]:
# Calculate the centroid of the outfield fence coordinates for each row
df_cleaned['fop_centroid'] = df_cleaned['fop'].apply(lambda coords: calculate_centroid(coords[1:]))

# Calculate the bearing between home plate and the centroid for each row
df_cleaned['field_orientation'] = df_cleaned.apply(lambda row: calculate_bearing(row['home_plate'], row['fop_centroid']), axis=1)

# Convert the bearing to a cardinal direction
df_cleaned['field_cardinal_direction'] = df_cleaned['field_orientation'].apply(degrees_to_cardinal_direction)



In [20]:
### Rename the Cleaned Dataframe back to the default name
df = df_cleaned

In [21]:
# ### In the fop, foul and home_plate columns, the coordinates are in the format (longitude, latitude).
# ### This is the opposite of the format that is used in Google Maps, so we need to reverse the order of the coordinates.
# df['fop'] = df['fop'].apply(lambda coords: [(coord[1], coord[0]) for coord in coords])
# df['foul'] = df['foul'].apply(lambda coords: [(coord[1], coord[0]) for coord in coords])
# df['home_plate'] = df['home_plate'].apply(lambda coord: (coord[1], coord[0]))


In [22]:
### Create a dataframe of just high school fields
hs_df = df[df['level'] == 'High School']

### Create a dataframe of all other fields
other_df = df[df['level'] != 'High School']




print(df.columns)

Index(['park_name', 'foul', 'fop', 'level', 'home_plate', 'foul_area_sqft',
       'fop_area_sqft', 'field_area_sqft', 'foul_area_per', 'fair_to_foul',
       'distances', 'max_distance', 'min_distance', 'avg_distance',
       'fop_centroid', 'field_orientation', 'field_cardinal_direction'],
      dtype='object')


In [23]:
hs_df.head()


,park_name,foul,fop,level,home_plate,foul_area_sqft,fop_area_sqft,field_area_sqft,foul_area_per,fair_to_foul,distances,max_distance,min_distance,avg_distance,fop_centroid,field_orientation,field_cardinal_direction
49,SW Christian HS Fort Worth - high school,"[(-97.4421601, 32.6418776), (-97.4431985, 32.6...","[(-97.4421601, 32.6418776), (-97.4421306000000...",High School,"(-97.4421601, 32.6418776)",37106.931904,169215.122852,206322.054756,0.179850,4.560202,"[314, 337, 371, 342, 354, 370, 362, 407, 413, ...",413,314,357.272727,"(-97.442824625, 32.64242071666667)",314.145471,NW
60,Soo High School,"[(-84.36583, 46.4813183), (-84.3658206, 46.482...","[(-84.36583, 46.4813183), (-84.3645398, 46.481...",High School,"(-84.36583, 46.4813183)",48045.311892,132184.170959,180229.482852,0.266579,2.751240,"[324, 353, 357, 370, 373, 359, 326]",373,324,351.714286,"(-84.3650061875, 46.4818190875)",48.561414,NE
61,Frankfort-Elberta Area HS,"[(-86.2258505, 44.6344082), (-86.2258357, 44.6...","[(-86.2258505, 44.6344082), (-86.2270454, 44.6...",High School,"(-86.2258505, 44.6344082)",35981.554245,121616.932836,157598.487081,0.228312,3.379980,"[310, 310, 316, 336, 360, 388, 384, 382, 381, ...",388,284,333.466667,"(-86.22663886875, 44.633879087500006)",226.676255,SW
62,Ironwood HS,"[(-90.1514578, 46.457154), (-90.1505766, 46.45...","[(-90.1514578, 46.457154), (-90.15234150000002...",High School,"(-90.1514578, 46.457154)",48494.539685,125485.905623,173980.445308,0.278736,2.587630,"[317, 327, 338, 327, 331, 336, 346, 361, 378, ...",380,308,344.120000,"(-90.15137326538463, 46.456347834615386)",175.868194,S
101,Morenci HS,"[(-84.2069811, 41.7223814), (-84.2058371, 41.7...","[(-84.2069811, 41.7223814), (-84.2071112, 41.7...",High School,"(-84.2069811, 41.7223814)",58086.513759,151942.087288,210028.601047,0.276565,2.615789,"[317, 318, 323, 338, 352, 359, 365, 369, 371, ...",373,313,347.217391,"(-84.206239, 41.72175010833334)",138.736309,SE


In [24]:
import pandas as pd
from fuzzywuzzy import fuzz, process

def find_best_match(school_name, choices, score_cutoff=80):
    best_match = process.extractOne(school_name, choices, scorer=fuzz.token_sort_ratio, score_cutoff=score_cutoff)
    if best_match:
        return best_match[0]
    else:
        return None

# Read CSV files
mhsaa_df = pd.read_csv('data/school_info/mhsaa_enrolment_2022.csv')
name_color_df = pd.read_csv('data\school_info\mhsaa_school_nickname_color_2020.csv')

# Get the list of park names from hs_df
park_names = hs_df['park_name'].tolist()

# Apply find_best_match function to create a new column 'best_match' in mhsaa_df
mhsaa_df['best_match'] = mhsaa_df['school_name'].apply(find_best_match, choices=park_names, score_cutoff=90)

## Pull the school_id, school_name, students, and division columns from mhsaa_df and add to hs_df
columns_to_extract = ['school_id', 'school_name', 'students', 'division']
for col in columns_to_extract:
    hs_df[col] = hs_df['park_name'].apply(lambda x: mhsaa_df.loc[mhsaa_df['best_match'] == x, col].iloc[0] if not mhsaa_df.loc[mhsaa_df['best_match'] == x, col].empty else None)

# Apply find_best_match function to create a new column 'best_match' in name_color_df
name_color_df['best_match'] = name_color_df['School'].apply(find_best_match, choices=park_names, score_cutoff=80)

## Pull the data from the name_color_df and add to hs_df (Nickname,Color1,Color2,Color3,Color4)
columns_to_extract = ['Nickname', 'Color1', 'Color2', 'Color3', 'Color4']
for col in columns_to_extract:
    hs_df[col] = hs_df['park_name'].apply(lambda x: name_color_df.loc[name_color_df['best_match'] == x, col].iloc[0] if not name_color_df.loc[name_color_df['best_match'] == x, col].empty else None)

# Drop the 'best_match' columns
# hs_df.drop(columns=['best_match'], inplace=True)

## Take a look at the new hs_df
hs_df.head()



# # Lookup the mhsaa_df['best_match'] in hs_df and return the columns: 'school_id', 'school_name', 'students', 'division'
# columns_to_extract = ['school_id', 'school_name', 'students', 'division']
# for col in columns_to_extract:
#     mhsaa_df[col] = mhsaa_df['best_match'].apply(lambda x: hs_df.loc[hs_df['park_name'] == x, col].iloc[0] if not hs_df.loc[hs_df['park_name'] == x, col].empty else None)

# # Apply find_best_match function to create a new column 'best_match' in name_color_df
# name_color_df['best_match'] = name_color_df['School'].apply(find_best_match, choices=park_names, score_cutoff=80)

# # Merge hs_df with mhsaa_df and name_color_df on the 'park_name' and 'best_match' columns
# hs_df = hs_df.merge(mhsaa_df, left_on='park_name', right_on='best_match', how='left', suffixes=('', '_from_mhsaa'))
# hs_df = hs_df.merge(name_color_df, left_on='park_name', right_on='best_match', how='left', suffixes=('', '_from_name_color'))

# # Drop the 'best_match' columns
# hs_df.drop(columns=['best_match_x', 'best_match_y'], inplace=True)


C:\Users\Justin\AppData\Local\Temp\ipykernel_13900\3874162956.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hs_df[col] = hs_df['park_name'].apply(lambda x: mhsaa_df.loc[mhsaa_df['best_match'] == x, col].iloc[0] if not mhsaa_df.loc[mhsaa_df['best_match'] == x, col].empty else None)
C:\Users\Justin\AppData\Local\Temp\ipykernel_13900\3874162956.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hs_df[col] = hs_df['park_name'].apply(lambda x: mhsaa_df.loc[mhsaa_df['best_match'] == x, col].iloc[0] if no

,park_name,foul,fop,level,home_plate,foul_area_sqft,fop_area_sqft,field_area_sqft,foul_area_per,fair_to_foul,...,field_cardinal_direction,school_id,school_name,students,division,Nickname,Color1,Color2,Color3,Color4
49,SW Christian HS Fort Worth - high school,"[(-97.4421601, 32.6418776), (-97.4431985, 32.6...","[(-97.4421601, 32.6418776), (-97.4421306000000...",High School,"(-97.4421601, 32.6418776)",37106.931904,169215.122852,206322.054756,0.179850,4.560202,...,NW,NaN,None,NaN,None,None,None,None,None,NaN
60,Soo High School,"[(-84.36583, 46.4813183), (-84.3658206, 46.482...","[(-84.36583, 46.4813183), (-84.3645398, 46.481...",High School,"(-84.36583, 46.4813183)",48045.311892,132184.170959,180229.482852,0.266579,2.751240,...,NE,NaN,None,NaN,None,None,None,None,None,NaN
61,Frankfort-Elberta Area HS,"[(-86.2258505, 44.6344082), (-86.2258357, 44.6...","[(-86.2258505, 44.6344082), (-86.2270454, 44.6...",High School,"(-86.2258505, 44.6344082)",35981.554245,121616.932836,157598.487081,0.228312,3.379980,...,SW,NaN,None,NaN,None,None,None,None,None,NaN
62,Ironwood HS,"[(-90.1514578, 46.457154), (-90.1505766, 46.45...","[(-90.1514578, 46.457154), (-90.15234150000002...",High School,"(-90.1514578, 46.457154)",48494.539685,125485.905623,173980.445308,0.278736,2.587630,...,S,NaN,None,NaN,None,Red Devils,Red,White,NaN,NaN
101,Morenci HS,"[(-84.2069811, 41.7223814), (-84.2058371, 41.7...","[(-84.2069811, 41.7223814), (-84.2071112, 41.7...",High School,"(-84.2069811, 41.7223814)",58086.513759,151942.087288,210028.601047,0.276565,2.615789,...,SE,NaN,None,NaN,None,Bulldogs,Maroon,White,NaN,NaN


In [25]:
### examin the output

hs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 494 entries, 49 to 722
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   park_name                 494 non-null    object 
 1   foul                      494 non-null    object 
 2   fop                       494 non-null    object 
 3   level                     494 non-null    object 
 4   home_plate                494 non-null    object 
 5   foul_area_sqft            494 non-null    float64
 6   fop_area_sqft             494 non-null    float64
 7   field_area_sqft           494 non-null    float64
 8   foul_area_per             494 non-null    float64
 9   fair_to_foul              494 non-null    float64
 10  distances                 494 non-null    object 
 11  max_distance              494 non-null    int64  
 12  min_distance              494 non-null    int64  
 13  avg_distance              494 non-null    float64
 14  fop_centr

In [26]:
#### ORIGINAL MATCHING CODE ####

# ########## TURNED OFF FOR NOW ###########
# #****** Don't need school info. want to test new fields

# #### MATCHING HIGH SCHOOL NAMES TO THE MHSAA DATA #####
# import pandas as pd
# from fuzzywuzzy import fuzz, process

# # Read the enrollment table from MHSAA website - 2022 enrollment
# mhsaa_df = pd.read_csv('data/school_info/mhsaa_enrolment_2022.csv')

# # Select just the high school level
# hs_df = df[df['level'] == 'high_school']
# other_df = df[df['level'] != 'high_school']

# # Set the threshold for the fuzzy match
# threshold = 90

# # Define a function to get the best fuzzy match with the threshold
# def get_best_match(field_name, school_names, threshold):
#     best_match = process.extractOne(field_name, school_names, scorer=fuzz.token_set_ratio)
#     if best_match[1] >= threshold:
#         return best_match[0]
#     return None

# # Apply the function to the 'field' column and store the result in a new 'match' column
# hs_df['match'] = hs_df['park_name'].apply(lambda x: get_best_match(x, mhsaa_df['school_name'], threshold))

# # #### This destroys a bunch of the data, at least every high school outside of michigan
# # # Drop rows with no match
# # # hs_df = hs_df.dropna(subset=['match'])

# # Lookup the hs_df['match'] in the mhsaa_df and return the columns: 'school_id', 'school_name', 'students', 'division'
# columns_to_extract = ['school_id', 'school_name', 'students', 'division']
# for col in columns_to_extract:
#     hs_df[col] = hs_df['match'].apply(lambda x: mhsaa_df.loc[mhsaa_df['school_name'] == x, col].iloc[0] if not mhsaa_df.loc[mhsaa_df['school_name'] == x, col].empty else None)



In [27]:

# merge the two dataframes back to a single one (using append)

# Merge the hs_df and other_df back together
merged_df = pd.concat([hs_df, other_df], ignore_index=True)

In [29]:
cleaned_df = merged_df


In [30]:
### Get Geolocation of each field based on home plate coordinates and return state and country
### This block takes a long time to run - will need to revisit
## up to ten minutes

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from tqdm import tqdm

geolocator = Nominatim(user_agent="baseball_field_locator")

# Function to get location information
def get_location_info(lng, lat):
    try:
        location = geolocator.reverse((lat, lng), timeout=10)
        state = location.raw['address'].get('state', None)
        country = location.raw['address'].get('country', None)
        return state, country
    except GeocoderTimedOut:
        print(f"GeocoderTimedOut error for coordinates: ({lng}, {lat})")
        return None, None
    except GeocoderServiceError:
        print(f"GeocoderServiceError for coordinates: ({lng}, {lat})")
        return None, None

# Extract the first coordinate for each field
df_cleaned['lng'], df_cleaned['lat'] = zip(*df_cleaned['home_plate'].apply(lambda x: x))

# Wrap the DataFrame apply function with tqdm for progress indication
tqdm.pandas(desc="Processing coordinates")

# Get state and country information for each field
df_cleaned[['state', 'country']] = df_cleaned.progress_apply(lambda row: get_location_info(row['lng'], row['lat']), axis=1, result_type='expand')


Processing coordinates:  15%|█▍        | 106/716 [00:52<05:02,  2.01it/s]

In [ ]:
# merged_df = df
merged_df.head(10)

# # Save the merged_df DataFrame as a JSON file
merged_df.to_json('data/default_updated_output.json', orient='records')


In [ ]:
column_list = merged_df.columns
print(column_list)

# ## Print a list of ten high school field names to test mascot lookup
# print(merged_df[merged_df['level'] == 'high_school']['park_name'].head(10).to_string())

In [ ]:
# ### Code to create report on the json file structure to be used as a reference later
# import json
# import pandas as pd
# from collections import defaultdict
# from builtins import list, dict

# # Load the JSON data from the file
# with open('data/updated_output_data.json', 'r') as file:
#     data = json.load(file)

# # Analyze the JSON data
# def analyze_structure(data, prefix=''):
#     structure = defaultdict(set)
    
#     if isinstance(data, dict):
#         for key, value in data.items():
#             new_prefix = f'{prefix}.{key}' if prefix else key
#             structure[new_prefix].add(type(value))
#             structure.update(analyze_structure(value, new_prefix))
#     elif isinstance(data, list):
#         for item in data:
#             structure.update(analyze_structure(item, prefix))
    
#     return structure

# # Generate the report
# structure = analyze_structure(data)
# descriptions = {
#     'field_name': 'The name of the baseball field.',
#     'foul': 'A list of coordinates representing the foul territory of the field. (lat, lon)',
#     'fop': 'A list of coordinates representing the fair territory of the field. (lat, lon)',
#     'level': 'The level of the field, e.g., high_school, college, etc.',
#     'home_plate': 'A list of coordinates representing the home plate location on the field. (lat, lon)',
#     'foul_area_sqft': 'The total area of the foul territory in square feet.',
#     'fop_area_sqft': 'The total area of the fair territory in square feet.',
#     'distances': 'A list of distances from home plate to the outfield fence at the vertices of the wall.',
#     'max_distance': 'The maximum distance from home plate to the outfield fence.',
#     'min_distance': 'The minimum distance from home plate to the outfield fence.',
#     'avg_distance': 'The average distance from home plate to the outfield fence.',
#     'fop_centroid': 'A list of coordinates representing the centroid of the fair territory.',
#     'field_orientation': "The angle (in degrees) of the field's orientation, with 0 degrees being North.",
#     'field_cardinal_direction': "The cardinal direction abbreviation (N, S, E, W, NE, NW, SE, SW) representing the field's orientation.",
#     'match': 'The matched school name found using fuzzy matching.',
#     'school_id': 'The unique identifier of the matched school.',
#     'school_name': 'The name of the matched school.',
#     'students': 'The number of students enrolled in the matched school.',
#     'division': 'The athletic division the matched school belongs to.'
# }

# # Replace <filename> with your desired filename without the extension
# filename = "output_data"

# def get_sample_value(data, key):
#     for item in data:
#         if key in item and item[key] is not None:
#             return item[key]
#     return None

# # Generate the report with sample values
# report = pd.DataFrame([(key, ', '.join([t.__name__ for t in types]), descriptions.get(key, ''), get_sample_value(data, key)) 
#                        for key, types in structure.items()],
#                       columns=['Key', 'Data Types', 'Description', 'Sample Value'])

# # Save the report as a CSV file
# report.to_csv(f"{filename}_report.csv", index=False)

# print(report)



In [ ]:
### Code to create report on some interesting stats like total fields, total area, ect

## Create a function to measure the total distance of the outside of each polygon
from shapely.geometry import Polygon
import pyproj

def calculate_perimeter(coords):

    perimeter = 0

    for i in range(len(coords)):
        x1, y1 = coords[i]
        x2, y2 = coords[(i + 1) % len(coords)]
        perimeter += math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

    return perimeter

# Get a sum of all of the perimeters
total_perimeter_fop = df['fop'].apply(calculate_perimeter).sum()
total_perimeter_foul = df['foul'].apply(calculate_perimeter).sum()




In [ ]:
import json

with open('data/default_updated_output.json', 'r') as file:
    data = json.load(file)



In [ ]:
# create a function to process the data, counting the orientations and filtering by level.

from collections import defaultdict

def process_data(data, level_filter=None):
    count_by_orientation = defaultdict(int)
    
    for record in data:
        if level_filter is None or record['level'] == level_filter:
            orientation = round(record['field_orientation'])
            count_by_orientation[orientation] += 1

    return count_by_orientation


In [ ]:
def create_polar_chart(data, num_bins=36, level_filter=None):
    count_by_orientation = process_data(data, level_filter)
    
    # Compute the histogram
    bin_edges = np.linspace(0.0, 2 * np.pi, num_bins + 1)
    bin_counts = np.zeros(num_bins)
    
    for orientation, count in count_by_orientation.items():
        idx = int(orientation / (360 / num_bins))
        if idx == num_bins:
            idx = 0
        bin_counts[idx] += count
    
    bin_width = 2 * np.pi / num_bins

    # Set plot size
    plt.figure(figsize=(10, 10))

    ax = plt.subplot(111, projection='polar')
    ax.set_theta_offset(np.pi / 2)

    ax.set_facecolor('#808080')
    ###
    # ax.set_yscale('log')

    
    # # Set dark background
    # ax.set_facecolor('#2b2b2b')
    plt.gca().set_rlabel_position(22.5)
    ax.set_ylim(-10, 100)  # Adjust based on max count

    bars = ax.bar(bin_edges[:-1], bin_counts, width=bin_width, bottom=-20)
    


    # Use custom colors and opacity
    for r, bar in zip(bin_counts, bars):
        bar.set_facecolor(plt.cm.viridis(r / max(bin_counts)))
        # bar.set_facecolor(plt.cm.plasma(r / max(bin_counts)))
        bar.set_alpha(0.8)

    plt.show()




In [ ]:
### UPDATED GPT CODE - LATE NIGHT FRIDAY


import numpy as np
import matplotlib.pyplot as plt

def create_polar_chart(data, num_bins=36, level_filter=None):
    count_by_orientation = process_data(data, level_filter)
    
    # Compute the histogram
    bin_edges = np.linspace(0.0, 2 * np.pi, num_bins + 1)
    bin_counts = np.zeros(num_bins)
    
    for orientation, count in count_by_orientation.items():
        idx = int(orientation / (360 / num_bins))
        if idx == num_bins:
            idx = 0
        bin_counts[idx] += count
    
    bin_width = 2 * np.pi / num_bins

    # Set plot size
    plt.figure(figsize=(10, 10))

    ax = plt.subplot(111, projection='polar')
    ax.set_theta_direction(-1)
    ax.set_theta_offset(np.pi / 2)

    ax.set_facecolor('#808080')
    ###
    # ax.set_yscale('log')

    
    # # Set dark background
    ax.set_facecolor('#2b2b2b')
    plt.gca().set_rlabel_position(22.5)
    ax.set_ylim(-20, 130)  # Adjust based on max count

    # Add bars for negative values
    zero_height_bars = ax.bar(bin_edges[:-1], np.abs(ax.get_ylim()[0]), width=bin_width, bottom=0.0, color='k', alpha=0.3)

    bars = ax.bar(bin_edges[:-1], bin_counts, width=bin_width, bottom=0)
    
    # Use custom colors and opacity
    for r, bar in zip(bin_counts, bars):
        bar.set_facecolor(plt.cm.viridis(r / max(bin_counts)))
        # bar.set_facecolor(plt.cm.plasma(r / max(bin_counts)))
        bar.set_alpha(0.8)

    plt.show()


In [ ]:
create_polar_chart(data, num_bins=50, level_filter=None)


#### GOAL
## fill the center portion of the plot to create a heat map of the field orientations



In [ ]:
create_polar_chart(data, num_bins=180, level_filter=None)


In [ ]:
### HEATMAP CODE

import numpy as np
import matplotlib.pyplot as plt

def create_heatmap(data, num_bins=36, level_filter=None):
    count_by_orientation = process_data(data, level_filter)

    # Compute the histogram
    bin_edges = np.linspace(0.0, 2 * np.pi, num_bins + 1)
    bin_counts = np.zeros(num_bins)

    for orientation, count in count_by_orientation.items():
        idx = int(orientation / (360 / num_bins))
        if idx == num_bins:
            idx = 0
        bin_counts[idx] += count

    # Reshape histogram data into a 2D array
    heatmap_data = np.tile(bin_counts, (num_bins, 1))

    # Set plot size
    plt.figure(figsize=(10, 10))

    # Create heatmap
    plt.imshow(heatmap_data, cmap='viridis', aspect='auto', interpolation='nearest', origin='lower')
    plt.colorbar(label='Counts')

    # Set x-axis ticks and labels
    plt.xticks(np.arange(0, num_bins, num_bins // 6), np.arange(0, 361, 60))
    plt.xlabel('Orientation (degrees)')

    # Set y-axis ticks and labels (assuming equal radial divisions)
    max_radius_label = 'Max Radius'
    plt.yticks(np.arange(0, num_bins, num_bins // 6), [0, 1, 2, 3, 4, max_radius_label])
    plt.ylabel('Radial Division')

    plt.show()

# Example usage:
# create_heatmap(data)


In [ ]:
# Example usage:
create_heatmap(data)

In [ ]:
# ### EXAMPLE# CODE FROM MATPLOTLIB GALLERY
# =======================
# Pie chart on polar axis
# =======================

# Demo of bar plot on a polar axis.
# """
# import numpy as np
# import matplotlib.pyplot as plt


# # Compute pie slices
# N = 20
# theta = np.linspace(0.0, 2 * np.pi, N, endpoint=False)
# radii = 10 * np.random.rand(N)
# width = np.pi / 4 * np.random.rand(N)

# ax = plt.subplot(111, projection='polar')
# bars = ax.bar(theta, radii, width=width, bottom=0.0)

# # Use custom colors and opacity
# for r, bar in zip(radii, bars):
#     bar.set_facecolor(plt.cm.viridis(r / 10.))
#     bar.set_alpha(0.5)

# plt.show()

In [ ]:
### Plot a histogram of the field orientations for all levels

import numpy as np
import matplotlib.pyplot as plt

def create_histogram(data, num_bins=36, level_filter=None):
    count_by_orientation = process_data(data, level_filter)
    
    # Compute the histogram
    bin_edges = np.linspace(0.0, 360.0, num_bins + 1)
    bin_counts = np.zeros(num_bins)
    
    for orientation, count in count_by_orientation.items():
        idx = int(orientation / (360 / num_bins))
        if idx == num_bins:
            idx = 0
        bin_counts[idx] += count
    
    bin_width = 360 / num_bins

    # Plot the histogram
    fig, ax = plt.subplots()
    ax.bar(bin_edges[:-1], bin_counts, width=bin_width, edgecolor='black')
    ax.set_xlabel('Field Orientation (degrees)')
    ax.set_ylabel('Number of Fields')
    ax.set_title('Field Orientation Histogram')

    # Set the major tick locations
    major_tick_locations = [45, 135, 225, 315]
    plt.xticks(major_tick_locations, major_tick_locations)


    
    plt.show()


In [ ]:
create_histogram(data, num_bins=36, level_filter=None)

END WORK BLOCK

In [ ]:
df['field_cardinal_direction'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
# Rename the dataframe back to the default name
df = merged_df

# Set the plot size
plt.figure(figsize=(10, 6))

# Create the scatter plot
plt.scatter(df_cleaned['min_distance'], df_cleaned['max_distance'], alpha=0.8)

# Customize the plot
plt.xlabel('Min Distance (feet)')
plt.ylabel('Max Distance (feet)')
plt.title('Scatter Plot of Min and Max Distances to Outfield Fence')

# Display the plot in the Jupyter Notebook
plt.show()


In [ ]:
##### CODE TO MAKE A LIST OF OUTLIERS #####

# Filter the DataFrame for fields with min distances below 100 feet
outliers = df[df['min_distance'] < 100]

# Display the outlier fields in the Jupyter Notebook
print(outliers[['park_name', 'min_distance', 'max_distance']])

# Save the outlier fields to a CSV file
outliers.to_csv('outlier_fields.csv', index=False)



In [ ]:
len(outliers)

In [ ]:
df.head()